**Внимание!**

- В задании были использованы предсказания за июнь 2016 года в качестве демонстрации, данные были выгружены из лучшего submission на kaggle, причём были использованы предсказания только на 1 час вперед (кроме 17-го часа последнего дня каждой зоны, там взяты предсказания на 6 часов вперед)
- К сожадению, мне не удалось разобраться с тем, как обрабатывать событие щелчка по интеративной карте, поэтому выбор зоны осуществляется с помощью dropdownlist'а, хоть я и понимаю, что это жутко неудобно. Для удобства тестирования варианты расположены от наиболее "интенсивной" зоны до наименее "интенсивной".
- Чтобы переключаться на карте между предсказанием и актуальными данными, нужно "отщёлкнуть" галочку в меню правого верхнего угла и щёлкнуть другую.

Приятных тестов ;)

In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [241]:
import folium
import branca.colormap as cm

In [3]:
from IPython.html import widgets
from IPython.display import display

## Выгрузим данные

In [80]:
valid_indexes = np.load('valid_indexes.npy')

In [249]:
pred_data = pd.read_csv('kaggle_week6.csv')
true_data = pd.read_pickle('true_table.pkl')

In [250]:
pred_data = pred_data[(pred_data['id'].apply(lambda s: s[-1]) == '1')
                      | (pred_data['id'].apply(lambda s: s.split('_')[1:3] == ['2016-06-30', '17']))]
pred_data['zone'] = pred_data['id'].apply(lambda s: s.split('_')[0]).astype(np.int)
pred_data['time'] = pd.to_datetime(pred_data['id'].apply(lambda s: s.split('_')[1]))\
+ pd.to_timedelta(pred_data['id'].apply(lambda s: s.split('_')[2]).astype(np.int), unit='h')\
+ pd.to_timedelta(pred_data['id'].apply(lambda s: s.split('_')[-1]).astype(np.int), unit='h')
pred_data['day'] = pred_data['time'].dt.day
pred_data['hour'] = pred_data['time'].dt.hour
pred_data.drop(columns=['id'], inplace=True)

In [123]:
sorted_indexes = true_data[['zone', 'y']].groupby('zone').mean().sort_values('y', ascending=False).index

## 1. Визуализация карт для определенного момента времени

In [181]:
NY_longmin = -74.25559
NY_longmax = -73.70001
NY_latmin = 40.49612
NY_latmax = 40.91553

In [239]:
def visualize_map(day, hour):
    
    # filering
    true_data_dm = true_data[(true_data['day'] == day)\
                             & (true_data['hour'] == hour)].drop(columns=['time', 'day',
                                                                          'hour']).set_index('zone').squeeze()
    pred_data_dm = pred_data[(pred_data['day'] == day)\
                             & (pred_data['hour'] == hour)].drop(columns=['time', 'day',
                                                                          'hour']).set_index('zone').squeeze()
    
    
    # making GeoJsonTooltip dict
    areas_dict = \
    {
        "type" : "FeatureCollection",
        "features" : []
    }

    lat_steps = np.linspace(NY_latmin, NY_latmax, 51)
    lon_steps = np.linspace(NY_longmin, NY_longmax, 51)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat_ind, lat in enumerate(lat_steps[:-1]):
        for lon_ind, lon in enumerate(lon_steps[:-1]):
            reg_id = lon_ind*50 + lat_ind + 1
            if reg_id in valid_indexes:

                # Определяем координаты углов для каждого квадрата
                upper_left = [lon, lat + lat_stride]
                upper_right = [lon + lon_stride, lat + lat_stride]
                lower_right = [lon + lon_stride, lat]
                lower_left = [lon, lat]

                # Определяем в dict
                coordinates = \
                [
                    upper_left,
                    upper_right,
                    lower_right,
                    lower_left,
                    upper_left
                ]

                area_dict = \
                {
                    "type" : "Feature",
                    "properties":
                    {
                        "id" : reg_id,
                        "true_value" : float(true_data_dm[reg_id]),
                        "pred_value" : float(pred_data_dm[reg_id])
                    },
                    "geometry" : 
                    {
                        "type": "Polygon",
                        "coordinates": [coordinates]
                    }
                }

                areas_dict["features"].append(area_dict)
                    
    
    # making map
    
    m = folium.Map(location=((NY_latmin + NY_latmax)/2, (NY_longmin + NY_longmax)/2), zoom_start=11)
    
    cmap = cm.linear.Reds_05.scale(0, max(true_data['y']))
    cmap.caption = "NY yellow taxi intensivity"
    
    true_feature_group = folium.FeatureGroup(name='Actual data')
    pred_feature_group = folium.FeatureGroup(name='Prediction', show=False)
    
    
    folium.GeoJson(
        areas_dict,
        style_function=lambda feature: 
        {
            'fillColor': cmap(feature["properties"]["true_value"]),
            'color': 'black',
            'weight': 0.35,
            'dashArray': '5, 5',
            'fillOpacity': 0.6
        },
        highlight_function=lambda feature: 
        {
            'weight': 2,
            'color':'blue'
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['id', 'true_value'],
            aliases=["region id", "intensivity"]
        )
    ).add_to(true_feature_group)
    
    folium.GeoJson(
        areas_dict,
        style_function=lambda feature: 
        {
            'fillColor': cmap(feature["properties"]["pred_value"]),
            'color': 'black',
            'weight': 0.35,
            'dashArray': '5, 5',
            'fillOpacity': 0.5
        },
        highlight_function=lambda feature: 
        {
            'weight': 3,
            'color':'blue'
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['id', 'pred_value'],
            aliases=["region id", "intensivity"]
        )
    ).add_to(pred_feature_group)

    true_feature_group.add_to(m)
    pred_feature_group.add_to(m)
    folium.LayerControl().add_to(m)
    cmap.add_to(m)
    
    display(m)

In [240]:
widday = widgets.IntSlider(min=1, max=30, description='Day: ')
widhour = widgets.IntSlider(max=23, description='Hour: ')
wint = widgets.interact(visualize_map, day=widday, hour=widhour)

interactive(children=(IntSlider(value=1, description='Day: ', max=30, min=1), IntSlider(value=0, description='…

## 2. Визуализация временных рядов для определенной зоны

In [133]:
def visualize_ts(zone):
    
    # filering
    true_data_zone = true_data[true_data['zone'] == zone]
    pred_data_zone = pred_data[pred_data['zone'] == zone]
    
    # visualizing
    plt.figure(figsize=(15, 10))
    plt.title('NY taxi orders intensivity in June 2016 (zone %s)'%zone, size=15)
    plt.plot(true_data_zone['time'], true_data_zone['y'], label='actual data')
    plt.plot(pred_data_zone['time'], pred_data_zone['y'], label='predicted data')
    plt.grid()
    plt.legend(prop={'size': 15})
    plt.show()

In [134]:
widzone = widgets.Dropdown(options=sorted_indexes, description='Zone: ')
wint = widgets.interact(visualize_ts, zone=widzone)

interactive(children=(Dropdown(description='Zone: ', options=(1282, 1232, 1181, 1231, 1281, 1230, 1180, 1233, …